### Downloading Tenforflow Pre-Trained Model From Tensorflow Model Zoo

In [5]:
import wget, object_detection, os

In [6]:
CUSTOM_MODEL_NAME = 'custom_ssd_mobilenet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [7]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('TensorFlow', 'workspace', 'Cylinder Detection','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','Cylinder Detection','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','Cylinder Detection','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace', 'Cylinder Detection','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
 }

In [8]:
files = {
    'PIPELINE_CONFIG':os.path.join('TensorFlow', 'workspace','Cylinder Detection','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [17]:
wget.download(PRETRAINED_MODEL_URL)      # Downloading the Pre-Trained Model                                     
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}     # Moving the Downloaded file to the 'Pre-Trained' Folder 
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}     # Extracting the File

'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar (1).gz'

### Image Annotation 

In [31]:
!labelimg

NOTE- Once the images are Anotated, they are moved to a 'train' and 'test folder.

### Creating Label Map  

In [71]:
labels = [{'name':'ThumbsUp', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### Generating TF Records

In [120]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 

NOTE - If the above command shows error, run it in cmd inside the script folder.
"python generate_tfrecord.py -x{'IMAGE Path'} -l{"LABEL_MAP Path"} -o{Annotations path with train.record'}

In [121]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

NOTE - If the above command shows error, run it in cmd inside the script folder.
"python generate_tfrecord.py -x{'IMAGE Path'} -l{"LABEL_MAP Path"} -o{Annotations path with test.record'}

### Extract the file of model which has been downloaded from Tensorflow Model Zoo and place the config folder to model  folder. Next we would update the model config as per the requiments.

# Training the Model

To train the model we would first copy the 'model_main_tf2.py' file from 'TensorFlow/models/research/object_detection' and past it in the Current Project folder inside 'Workspace'.

Then we will cd to the main directory and execute the below command:

In [ ]:
python TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=TensorFlow/workspace/"Cylinder Detection"/models/custom_ssd_mobilenet --pipeline_config_path=TensorFlow/workspace/"Cylinder Detection"/models/custom_ssd_mobilenet/pipeline.config --num_train_steps=2000

This would train the model and the checkpoints created will be saved in custom model folder.

# Loading the Model

In [9]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2
import numpy as np

In [10]:
# Loading config
conf = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
my_model = model_builder.build(model_config=conf['model'], is_training=False)

# Restoring Checkpoint
cp = tf.compat.v2.train.Checkpoint(model=my_model)
cp.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = my_model.preprocess(image)
    prediction_dict = my_model.predict(image, shapes)
    detections = my_model.postprocess(prediction_dict, shapes)
    return detections

# Detecting Cylinder

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [16]:
cap = cv2.VideoCapture("plant_2.mp4")
cap.set(3,1080)
cap.set(4,720)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    frame = frame [50:1000, 50:1000]
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break